In [2]:
import numpy as np 

## La classe abstraite Layer 
On commence par définir la classe abstraite Layer qui contient les caractéristiques qui seront communes à toutes les couches. L'entrée, la sortie la fonction pour la propagation et celle pour la rétropropagation..

In [1]:
class Layer:
    def __init__(self):
        self.input = None
        sel.output = None 
        
    # Donne la sortie Y d'une couche en ayant l'entrée X 
    def forward_propagation(self, input):
        raise NotImplementedError
        
    def backward_propagation(self,output_error,learning_rate):
        raise NotImplementedError

## Backward propagation
Les formules fondamentales de la partie linéaire sont les suivantes : 

$$
\begin{aligned}
&\frac{\partial E}{\partial X}=\frac{\partial E}{\partial Y} W^{t} \\
&\frac{\partial E}{\partial W}=X^{t} \frac{\partial E}{\partial Y} \\
&\frac{\partial E}{\partial B}=\frac{\partial E}{\partial Y}
\end{aligned}
$$


In [19]:
class FCLayer(Layer):
    #input size: nombre de neurones en entrées 
    #output size: nombre de neurones en sortie 
    def __init__(self,input_size,output_size):
        self.weights = np.random.rand(input_size,output_size)-0.5 #les poids aléatoirement pris 
        self.bias = np.random.rand(1,output_size)-0.5 #le biais initialisés aléatoirements 
        
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input,self.weights) + self.bias #notation matricielle 
        return self.output
    
    #calcul de DE/DW, DE/DB pour une erreur d'entrée donnée DE/DY 
    #ici output_error est la dérivée de l'erreur par rapport à la sortie 
    #on code tout par rapport à une couche ! 
    def backward_propagation(self,output_error,learning_rate):
        input_error = np.dot(output_error,self.weights.T)
        weigths_error = np.dot(self.input.T,output_error)
        
        self.weights -= learning_rate*weigths_error
        self.bias -= learning_rate*output_error
        return input_error #c'est aussi le output error de la couche précédente ! 
        

## Couche d'activation 
Il faut appliquer des fonctions non linéaires en sortie de certaines couches pour pouvoir permettre au réseau d'apprendre des choses... 

In [5]:
class ActivationLayer(Layer):
    def __init__(self,activation,activation_prime):
        self.activation=activation
        self.activation_prime = activation_prime
        
    def forward_propagation(self,input_data):
        self.input = input_data
        self.output=self.activation(self.input) #activation pour donner la sortie 
        return self.output 
    def backward_propagation(self,output_error,learning_rate):
        return self.activation_prime(self.input)*output_error
        

## On écrit à côté quelques fonctions d'activation 

In [7]:
import numpy as np
def tanh(x):
    return np.tanh(x);

def tanh_prime(x):
    return 1-np.tanh(x)**2;

## On donne aussi la fonction de perte 

In [9]:
def mse(y_true,y_pred):
    return(np.mean(np.power(y_true-y_pred,2)))
def mse_prime(y_true,y_pred):
    return 2*(y_pred-y_true)/y_true.size;

## La classe du réseau de neurone 

In [25]:
class Network:
    def __init__(self):
        self.layers=[]
        self.loss= None 
        self.loss_prime = None #pas de fonction de perte ici, ni de couches
        
    #ajouter une couche au réseau 
    
    def add(self,layer):
        self.layers.append(layer)
        
    def use(self,loss,loss_prime):
        self.loss=loss
        self.loss_prime=loss_prime
        
    #la fonction pour prédire 
    
    def predict(self,input_data):
        samples = len(input_data)
        result = []
        #on prédit les sorties pour chaque entrée 
        
        for i in range(samples):
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)
        return result
    
    #entrainer le réseau de neurones 
    
    def fit(self,x_train,y_train,epochs,learning_rate):
        samples = len(x_train)
        #boucle d'entrainement 
        
        for i in range(epochs):
            err = 0
            for j in range(samples):
                #forward propagation 
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)
                err+=self.loss(y_train[j],output)
                
                #backpropagation 
                error = self.loss_prime(y_train[j],output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error,learning_rate)
            err/=samples
            print('epoch %d/%d error = %f' %(i+1,epochs,err))

In [27]:
# training data
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

# network
net = Network()
net.add(FCLayer(2, 3))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(3, 1))
net.add(ActivationLayer(tanh, tanh_prime))

# train
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=1000, learning_rate=0.1)



epoch 1/1000 error = 0.386365
epoch 2/1000 error = 0.319516
epoch 3/1000 error = 0.305605
epoch 4/1000 error = 0.301074
epoch 5/1000 error = 0.298983
epoch 6/1000 error = 0.297753
epoch 7/1000 error = 0.296896
epoch 8/1000 error = 0.296226
epoch 9/1000 error = 0.295662
epoch 10/1000 error = 0.295163
epoch 11/1000 error = 0.294707
epoch 12/1000 error = 0.294283
epoch 13/1000 error = 0.293882
epoch 14/1000 error = 0.293501
epoch 15/1000 error = 0.293136
epoch 16/1000 error = 0.292785
epoch 17/1000 error = 0.292447
epoch 18/1000 error = 0.292120
epoch 19/1000 error = 0.291804
epoch 20/1000 error = 0.291498
epoch 21/1000 error = 0.291201
epoch 22/1000 error = 0.290913
epoch 23/1000 error = 0.290634
epoch 24/1000 error = 0.290362
epoch 25/1000 error = 0.290099
epoch 26/1000 error = 0.289843
epoch 27/1000 error = 0.289594
epoch 28/1000 error = 0.289352
epoch 29/1000 error = 0.289117
epoch 30/1000 error = 0.288889
epoch 31/1000 error = 0.288667
epoch 32/1000 error = 0.288451
epoch 33/1000 err

epoch 437/1000 error = 0.279197
epoch 438/1000 error = 0.279195
epoch 439/1000 error = 0.279194
epoch 440/1000 error = 0.279192
epoch 441/1000 error = 0.279191
epoch 442/1000 error = 0.279190
epoch 443/1000 error = 0.279188
epoch 444/1000 error = 0.279187
epoch 445/1000 error = 0.279186
epoch 446/1000 error = 0.279184
epoch 447/1000 error = 0.279183
epoch 448/1000 error = 0.279182
epoch 449/1000 error = 0.279180
epoch 450/1000 error = 0.279179
epoch 451/1000 error = 0.279178
epoch 452/1000 error = 0.279176
epoch 453/1000 error = 0.279175
epoch 454/1000 error = 0.279174
epoch 455/1000 error = 0.279173
epoch 456/1000 error = 0.279171
epoch 457/1000 error = 0.279170
epoch 458/1000 error = 0.279169
epoch 459/1000 error = 0.279168
epoch 460/1000 error = 0.279166
epoch 461/1000 error = 0.279165
epoch 462/1000 error = 0.279164
epoch 463/1000 error = 0.279163
epoch 464/1000 error = 0.279162
epoch 465/1000 error = 0.279160
epoch 466/1000 error = 0.279159
epoch 467/1000 error = 0.279158
epoch 46

epoch 889/1000 error = 0.278920
epoch 890/1000 error = 0.278919
epoch 891/1000 error = 0.278919
epoch 892/1000 error = 0.278919
epoch 893/1000 error = 0.278918
epoch 894/1000 error = 0.278918
epoch 895/1000 error = 0.278918
epoch 896/1000 error = 0.278918
epoch 897/1000 error = 0.278917
epoch 898/1000 error = 0.278917
epoch 899/1000 error = 0.278917
epoch 900/1000 error = 0.278917
epoch 901/1000 error = 0.278916
epoch 902/1000 error = 0.278916
epoch 903/1000 error = 0.278916
epoch 904/1000 error = 0.278916
epoch 905/1000 error = 0.278915
epoch 906/1000 error = 0.278915
epoch 907/1000 error = 0.278915
epoch 908/1000 error = 0.278915
epoch 909/1000 error = 0.278914
epoch 910/1000 error = 0.278914
epoch 911/1000 error = 0.278914
epoch 912/1000 error = 0.278914
epoch 913/1000 error = 0.278913
epoch 914/1000 error = 0.278913
epoch 915/1000 error = 0.278913
epoch 916/1000 error = 0.278913
epoch 917/1000 error = 0.278912
epoch 918/1000 error = 0.278912
epoch 919/1000 error = 0.278912
epoch 92

In [28]:

out = net.predict(x_train)
print(out)

[array([[0.52422408]]), array([[0.52731055]]), array([[0.51468488]]), array([[0.51782252]])]
